In [2]:
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.functions import year, col, to_date

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import Imputer

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('Motor Vehicle Collisions Analysis') \
    .getOrCreate()

In [4]:
data_path = 'Motor_Vehicle_Collisions_-_Crashes.csv'
df = spark.read.csv(data_path, header=True, inferSchema=True)
df.printSchema()

root
 |-- CRASH DATE: string (nullable = true)
 |-- CRASH TIME: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- ZIP CODE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- ON STREET NAME: string (nullable = true)
 |-- CROSS STREET NAME: string (nullable = true)
 |-- OFF STREET NAME: string (nullable = true)
 |-- NUMBER OF PERSONS INJURED: string (nullable = true)
 |-- NUMBER OF PERSONS KILLED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: integer (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: integer (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: string (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: string (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: integer (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 2: strin

In [5]:
# Menampilkan semua kolom
df.select("*").show()

+----------+----------+---------+--------+---------+----------+--------------------+--------------------+--------------------+--------------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+--------------------+--------------------+-------------------+-------------------+-------------------+
|CRASH DATE|CRASH TIME|  BOROUGH|ZIP CODE| LATITUDE| LONGITUDE|            LOCATION|      ON STREET NAME|   CROSS STREET NAME|     OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLED|CONTRIBUTING

In [6]:
for column in df.columns:
    unique_count = df.select(column).distinct().count()
    print(f"{column}: {unique_count} unique values")

CRASH DATE: 4341 unique values
CRASH TIME: 1441 unique values
BOROUGH: 7 unique values
ZIP CODE: 237 unique values
LATITUDE: 126786 unique values
LONGITUDE: 98476 unique values
LOCATION: 285153 unique values
ON STREET NAME: 18480 unique values
CROSS STREET NAME: 20267 unique values
OFF STREET NAME: 228344 unique values
NUMBER OF PERSONS INJURED: 34 unique values
NUMBER OF PERSONS KILLED: 8 unique values
NUMBER OF PEDESTRIANS INJURED: 15 unique values
NUMBER OF PEDESTRIANS KILLED: 5 unique values
NUMBER OF CYCLIST INJURED: 7 unique values
NUMBER OF CYCLIST KILLED: 5 unique values
NUMBER OF MOTORIST INJURED: 33 unique values
NUMBER OF MOTORIST KILLED: 7 unique values
CONTRIBUTING FACTOR VEHICLE 1: 62 unique values
CONTRIBUTING FACTOR VEHICLE 2: 62 unique values
CONTRIBUTING FACTOR VEHICLE 3: 52 unique values
CONTRIBUTING FACTOR VEHICLE 4: 42 unique values
CONTRIBUTING FACTOR VEHICLE 5: 31 unique values
COLLISION_ID: 2089381 unique values
VEHICLE TYPE CODE 1: 1658 unique values
VEHICLE TY

In [8]:
# Mengonversi kolom 'CRASH DATE' menjadi tipe Date dengan format yang sesuai
df = df.withColumn("CRASH DATE", to_date(col("CRASH DATE"), "dd/MM/yyyy"))

# Menambahkan kolom tahun dengan menggunakan fungsi year
df = df.withColumn("YEAR", year(col("CRASH DATE")))

In [9]:
selected_columns = ['YEAR','COLLISION_ID', 'BOROUGH','NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1']
df_selected = df.select(selected_columns)
df_selected.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- COLLISION_ID: integer (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- NUMBER OF PERSONS INJURED: string (nullable = true)
 |-- NUMBER OF PERSONS KILLED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: integer (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: integer (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: string (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: string (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: integer (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = true)



In [10]:
# Summing up the injury and fatality related columns
df_aggregated = df_selected.withColumn('Total Injured', 
                              col('NUMBER OF PERSONS INJURED') + 
                              col('NUMBER OF PEDESTRIANS INJURED') + 
                              col('NUMBER OF CYCLIST INJURED') + 
                              col('NUMBER OF MOTORIST INJURED')) \
                  .withColumn('Total Killed', 
                              col('NUMBER OF PERSONS KILLED') + 
                              col('NUMBER OF PEDESTRIANS KILLED') + 
                              col('NUMBER OF CYCLIST KILLED') + 
                              col('NUMBER OF MOTORIST KILLED'))

In [11]:
selected_columns = ['YEAR','COLLISION_ID', 'BOROUGH', 'Total Killed', 'Total Injured','CONTRIBUTING FACTOR VEHICLE 1']
df = df_aggregated.select(selected_columns)
df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- COLLISION_ID: integer (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- Total Killed: double (nullable = true)
 |-- Total Injured: double (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = true)



In [12]:
# Handle null values
imputer = Imputer(inputCols=["Total Killed", "Total Injured"], 
                  outputCols=["Total Killed", "Total Injured"])
df = imputer.fit(df).transform(df)

In [13]:
# Assemble features
vecAssembler = VectorAssembler(inputCols=["Total Killed", "Total Injured"], outputCol="features")
df_kmeans = vecAssembler.transform(df)

In [14]:
# KMeans model with 3 clusters
kmeans = KMeans().setK(3).setSeed(1)
model = kmeans.fit(df_kmeans)

In [15]:
# Predictions
predictions = model.transform(df_kmeans)

In [16]:
output = predictions.toPandas() 

In [17]:
output.head()

,YEAR,COLLISION_ID,BOROUGH,Total Killed,Total Injured,CONTRIBUTING FACTOR VEHICLE 1,features,prediction
0,2021.0,4455765.0,None,0.0,4.0,Aggressive Driving/Road Rage,"[0.0, 4.0]",1
1,NaN,4513547.0,None,0.0,2.0,Pavement Slippery,"[0.0, 2.0]",1
2,NaN,4541903.0,None,0.0,0.0,Following Too Closely,"(0.0, 0.0)",0
3,2021.0,4456314.0,BROOKLYN,0.0,0.0,Unspecified,"(0.0, 0.0)",0
4,NaN,4486609.0,BROOKLYN,0.0,0.0,None,"(0.0, 0.0)",0


In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x="YEAR", y="count", hue="prediction", data=df_pandas)
plt.title('Total Accident Categories per Year')
plt.xlabel('Year')
plt.ylabel('Total Accidents')
plt.legend(title='Category', labels=['Light', 'Moderate', 'Severe'])
plt.show()

NameError: name 'plt' is not defined